In [1]:
import torch
import os
import numpy as np
from PIL import Image
import pandas as pd
import torchvision
import torch.nn as nn
import torchvision.models as models
from torchvision import transforms
from torch.utils.data import TensorDataset, DataLoader, dataset

import numpy as np
import math
import cv2
import matplotlib.pyplot as plt
#plt.rcParams['figure.figsize'] = [12, 30]
#plt.rcParams['figure.dpi'] = 100



device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device is", device)

C:\Users\Anton\AppData\Local\Temp\ipykernel_14628\2010194887.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


device is cpu


In [2]:
class CaptionDataset(torch.utils.data.Dataset):
    def __init__(self, dir):
        self.image_dir = os.path.join(dir,"Images")
        self.df = pd.read_csv(os.path.join(dir,"captions.txt"))

    def __getitem__(self,i):
        caption = self.df["caption"][i]
        image_path = os.path.join(self.image_dir,self.df["image"][i])
        image = transforms.ToTensor()(Image.open(image_path))
        return (image,caption)

    def __len__(self):
        return len(self.df)

In [3]:
#EXAMPLE/TEST TO BE DELETED
ds = CaptionDataset("archive")
data_loader = DataLoader(ds)
it = iter(data_loader)

In [4]:
class DecoderRNN(nn.Module):
    def __init__(self, embed_size, hidden_size, vocab_size, num_layers=1):
        super(DecoderRNN, self).__init__()
        
        # define the properties
        self.embed_size = embed_size
        self.hidden_size = hidden_size
        self.vocab_size = vocab_size
        
        # lstm cell
        self.lstm_cell = nn.LSTMCell(input_size=embed_size, hidden_size=hidden_size)
    
        # output fully connected layer
        self.fc_out = nn.Linear(in_features=self.hidden_size, out_features=self.vocab_size)
    
        # embedding layer
        self.embed = nn.Embedding(num_embeddings=self.vocab_size, embedding_dim=self.embed_size)
    
        # activations
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, features, captions):
        
        # batch size
        batch_size = features.size(0)
        
        # init the hidden and cell states to zeros
        hidden_state = torch.zeros((batch_size, self.hidden_size)).cuda()
        cell_state = torch.zeros((batch_size, self.hidden_size)).cuda()
    
        # define the output tensor placeholder
        outputs = torch.empty((batch_size, captions.size(1), self.vocab_size)).cuda()

        # embed the captions
        captions_embed = self.embed(captions)
        
        # pass the caption word by word
        for t in range(captions.size(1)):

            # for the first time step the input is the feature vector
            if t == 0:
                hidden_state, cell_state = self.lstm_cell(features, (hidden_state, cell_state))
                
            # for the 2nd+ time step, using teacher forcer
            else:
                hidden_state, cell_state = self.lstm_cell(captions_embed[:, t, :], (hidden_state, cell_state))
            
            # output of the attention mechanism
            out = self.fc_out(hidden_state)
            
            # build the output tensor
            outputs[:, t, :] = out
    
        return outputs

In [5]:
from torchsummary import summary

class Linear(nn.Module):
    def __init__(self):
        super().__init__()
        self.last_linear = nn.Linear(-1,1000)
        
    def forward(self, x):
        x = self.last_linear(x)
        return x

# Use a predefined model as a feature extractor
resnet = models.resnet18(pretrained=True)

#Freeze parameters
#for param in resnet.parameters():
#    param.requires_grad = False

#Replace connected layer
#resnet.fc = (Linear())

summary(resnet, (3, 224, 224))


C:\Users\Anton\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Anton\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


RuntimeError: Trying to create tensor with negative dimension -1: [1000, -1]

In [ ]:
losses = list()
val_losses = list()

for epoch in range(1, 10+1):

    for i_step in range(1, total_step+1):

        #zero the gradients
        decoder.zero_grad()
        encoder.zero_grad()

        # set decoder and encoder into train mode
        encoder.train()
        decoder.train()

        #radnomly sample a caption lengt, and sample indices with that length
        indices = train_data_loader.dataset.get_train_indices()

        # Create and assign a batch sampler to retrieve a batch with the sampled indices.
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        train_data_loader.batch_sampler.sampler = new_sampler
        
        # Obtain the batch.
        images, captions = next(iter(train_data_loader))
        
        # make the captions for targets and teacher forcer
        captions_target = captions[:, 1:].to(device)
        captions_train = captions[:, :captions.shape[1]-1].to(device)

        # Move batch of images and captions to GPU if CUDA is available.
        images = images.to(device)
        
        # Pass the inputs through the CNN-RNN model.
        features = encoder(images)
        outputs = decoder(features, captions_train)
        
        # Calculate the batch loss
        loss = criterion(outputs.view(-1, vocab_size), captions_target.contiguous().view(-1))
        
        # Backward pass
        loss.backward()
        
        # Update the parameters in the optimizer
        optimizer.step()
        
        # - - - Validate - - -
        # turn the evaluation mode on
        with torch.no_grad():
            
            # set the evaluation mode
            encoder.eval()
            decoder.eval()

            # get the validation images and captions
            val_images, val_captions = next(iter(val_data_loader))

            # define the captions
            captions_target = val_captions[:, 1:].to(device)
            captions_train = val_captions[:, :val_captions.shape[1]-1].to(device)

            # Move batch of images and captions to GPU if CUDA is available.
            val_images = val_images.to(device)

            # Pass the inputs through the CNN-RNN model.
            features = encoder(val_images)
            outputs = decoder(features, captions_train)

            # Calculate the batch loss.
            val_loss = criterion(outputs.view(-1, vocab_size), captions_target.contiguous().view(-1))
        
        # append the validation loss and training loss
        val_losses.append(val_loss.item())
        losses.append(loss.item())
        
        # save the losses
        np.save('losses', np.array(losses))
        np.save('val_losses', np.array(val_losses))
        
        # Get training statistics.
        stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Val Loss: %.4f' % (epoch, num_epochs, i_step, total_step, loss.item(), val_loss.item())
        
        # Print training statistics (on same line).
        print('\r' + stats, end="")
        sys.stdout.flush()
            
    # Save the weights.
    if epoch % save_every == 0:
        print("\nSaving the model")
        torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-%d.pth' % epoch))
        torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-%d.pth' % epoch))

In [ ]:
transforms = transforms.Compose([
    # Add training augmentations here, remember: we do not want to transform the validation images.
    # For information about augmentation see: https://pytorch.org/vision/stable/transforms.html
    transforms.ToPILImage(),
    transforms.Resize(size=(224,224)),
    #transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
def train(encoder, decoder, optimizer, train_loader, val_loader, epochs, tokenizer, criterion):
    for epoch in range(epochs):
        print("Starting epoch",epoch,"/",epochs)
        train_loss_sum=0
        train_loss_num=0
        for batch_nr, (image, caption) in enumerate(train_loader):
            if(batch_nr%20):
                print("Processing batch:",batch_nr)
            images = images.to(device)
            caption = caption.to(device)
            features = encoder(image)
            prediction = decoder(features)
            caption_tokens = tokenizer(caption)
            optimizer.zero_grad()
            loss = criterion(prediction,caption_tokens)
            train_loss_sum+=loss.item()
            train_loss_num+=1
            loss.backward()
            optimizer.step()

        val_loss_sum=0
        val_loss_num=0
        for batch_nr, (image,caption) in enumerate(val_loader):
            images = images.to(device)
            caption = caption.to(device)
            features = encoder(image)
            prediction = decoder(features)
            caption_tokens = tokenizer(caption)
            loss = criterion(prediction,caption_tokens)
            val_loss_sum+=loss.item()
            val_loss_num+=1

        print("Train loss:",train_loss_sum/train_loss_num,"\t Val loss:",val_loss_sum/val_loss_num)

            


In [2]:
reviews = ["This is a normal test sentence. Please do not use it in vain for this is not it's intended use.","This is shit"]
import torchtext

tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
vocab = torchtext.vocab.build_vocab_from_iterator(
    tokenizer(review) for review in reviews
)

print(vocab.get_stoi())

tokenized_reviews = [tokenizer(review) for review in reviews]
numericalized_reviews = [[vocab[token] for token in review] for review in tokenized_reviews]
print(tokenized_reviews)
print(numericalized_reviews)
np.array(numericalized_reviews)
t = torch.from_numpy(np.array(numericalized_reviews))

{'is': 0, '.': 2, 'this': 1, 'it': 3, 'not': 4, 'a': 7, 'use': 5, "'": 6, 'sentence': 15, 'do': 8, 'for': 9, 'in': 10, 'intended': 11, 'normal': 12, 'please': 13, 's': 14, 'shit': 16, 'test': 17, 'vain': 18}
[['this', 'is', 'a', 'normal', 'test', 'sentence', '.', 'please', 'do', 'not', 'use', 'it', 'in', 'vain', 'for', 'this', 'is', 'not', 'it', "'", 's', 'intended', 'use', '.'], ['this', 'is', 'shit']]
[[1, 0, 7, 12, 17, 15, 2, 13, 8, 4, 5, 3, 10, 18, 9, 1, 0, 4, 3, 6, 14, 11, 5, 2], [1, 0, 16]]


NameError: name 'np' is not defined